In [13]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from datetime import datetime # datetime library to handle dates in datasets

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
CLIENT_ID = "MCV5NY4BQNKONYBRPPEJPIKGT10LSZOD3LU0ZXOKHF5NNE3Y"
CLIENT_SECRET = "AZ021VIEHWYW2ON03AP32HSQY5GMWAE1YHMRU1NP3JXMJYIR"
VERSION = "20170410"
# @hidden_cell

LIMIT = 30

In [31]:
def search_address_df(address, SEARCH_QUERY):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    LATITUDE = location.latitude
    LONGITUDE = location.longitude
    url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, LATITUDE, LONGITUDE, VERSION, SEARCH_QUERY, LIMIT)
    results = requests.get(url).json()
    venues = results["response"]["venues"]
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'url'] + [col for col in dataframe.columns if col.startswith('location.')]
    dataframe_filtered = dataframe.ix[:, filtered_columns]
    dataframe_filtered.columns = [column.split(".")[-1] for column in dataframe_filtered.columns]
    return dataframe_filtered.head()

search_address_df("525 Stockton St, San Francisco CA", "tea")

,name,url,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state
0,Samovar Tea Lounge,http://samovarlife.com,730 Howard St,US,San Francisco,United States,btwn 3rd & 4th St,853,"[730 Howard St (btwn 3rd & 4th St), San Franci...","[{'lng': -122.4022650718689, 'lat': 37.7842827...",37.784283,-122.402265,94103,CA
1,Peet's Coffee & Tea,http://t.co/a0Sgjn9fPF,595 Mission St,US,San Francisco,United States,at Second St,779,"[595 Mission St (at Second St), San Francisco,...","[{'lng': -122.3993058, 'lat': 37.78808019, 'la...",37.788080,-122.399306,94105,CA
2,Peet's Coffee & Tea,http://t.co/a0Sgjn9fPF,450 Sansome St #1,US,San Francisco,United States,at Clay St,704,"[450 Sansome St #1 (at Clay St), San Francisco...","[{'lng': -122.401276, 'lat': 37.7947409, 'labe...",37.794741,-122.401276,94111,CA
3,Peet's Coffee & Tea,http://t.co/a0Sgjn9fPF,2 Embarcadero Ctr Ste R2113,US,San Francisco,United States,at Front St,875,"[2 Embarcadero Ctr Ste R2113 (at Front St), Sa...","[{'lng': -122.3989309, 'lat': 37.79475517, 'la...",37.794755,-122.398931,94111,CA
4,Asha Tea House,http://www.ashateahouse.com,17 Kearny St,US,San Francisco,United States,Market Street,464,"[17 Kearny St (Market Street), San Francisco, ...","[{'lng': -122.40342442357135, 'lat': 37.788081...",37.788081,-122.403424,94108,CA


In [14]:
plotly.tools.set_credentials_file(username='LeouOn', api_key='Q7diFE7K5zp9wDSao4lh') # plotly credentials
mapbox_access_token = 'pk.eyJ1IjoiYWxleGFrbHNvbiIsImEiOiJjajFwZnhqYW0wMDBqMzJrNWVhbmtuY2x1In0.lIla_DRBTcVco97HYIY_Ow' # Mapbox token

def search_address_plot(address, SEARCH_QUERY):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    LATITUDE = location.latitude
    LONGITUDE = location.longitude
    url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, LATITUDE, LONGITUDE, VERSION, SEARCH_QUERY, LIMIT)
    results = requests.get(url).json()
    venues = results["response"]["venues"]
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'url'] + [col for col in dataframe.columns if col.startswith('location.')]
    dataframe_filtered = dataframe.ix[:, filtered_columns]
    dataframe_filtered.columns = [column.split(".")[-1] for column in dataframe_filtered.columns]    
    data = Data(
        [
            Scattermapbox(
                lat=[LATITUDE] + list(dataframe_filtered.lat),
                lon=[LONGITUDE] + list(dataframe_filtered.lng),
                mode='markers',
                marker=Marker(
                    size=10,
                    color=["red"] + ["blue"]*len(dataframe_filtered)
                ),
                text=["Home"] + list(dataframe_filtered.name),
            )
        ]
    )

    # define the layout and centre the map around Galvanize
    layout = Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=int(LATITUDE),
                lon=int(LONGITUDE)
            ),
            pitch=0,
            zoom=5
        ),
    )

    # plot the data
    fig = dict(data=data, layout=layout)
    return py.iplot(fig, filename='Galvanize Search Mapbox')
search_address_plot("44 Tehama St San Francisco, CA", "chinese")

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~LeouOn/0 or inside your plot.ly account where it is named 'Galvanize Search Mapbox'


In [1]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_09bd790c49f84532a1fc326f9db6c10c(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_e345ae217ed64e5c5dddb085b0fc39c33c619e6a','domain': {'id': 'fc6742b04ad74b7bb64a3873d7e5a050'},
            'password': 'rs3H.,I?&B?)0fMN'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_09bd790c49f84532a1fc326f9db6c10c('DefaultProjectyuneleougmailcom', 'weightsteps.csv'))

In [2]:
df_data_2 = pd.read_csv(get_object_storage_file_with_credentials_09bd790c49f84532a1fc326f9db6c10c('DefaultProjectyuneleougmailcom', 'ExerciseLog.csv'))
df_data_2 = df_data_2.rename(columns={'occurredDate':'Date'})
df_data_2 = df_data_2.set_index('Date')

In [29]:
df_data_3 = pd.read_csv(get_object_storage_file_with_credentials_09bd790c49f84532a1fc326f9db6c10c('DefaultProjectyuneleougmailcom', 'ConvLog_flagged.csv'))
df_data_3 = df_data_3.rename(columns={'occurredDate':'Date'})
df_data_3 = df_data_3.set_index('Date')

In [3]:
import json
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(username='f1851a68-b758-4c1a-a04f-74d8a1b91349',
   password='cA1gfBAb6CfM',
   version='2016-05-19')

In [4]:
def analyze_tones(df, row):
    df3 = pd.DataFrame(index = ['Date'], columns = ['notes', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Analytical',
       'Confident', 'Tentative', 'Openness', 'Conscientiousness',
       'Extraversion', 'Agreeableness', 'Emotional Range'])
    output = tone_analyzer.tone(text=df['notes'][row])
    emotional_tone = pd.DataFrame(output['document_tone']['tone_categories'][0]['tones']).set_index('tone_name').drop('tone_id', 1)
    language_tone = pd.DataFrame(output['document_tone']['tone_categories'][1]['tones']).set_index('tone_name').drop('tone_id', 1)
    social_tone = pd.DataFrame(output['document_tone']['tone_categories'][2]['tones']).set_index('tone_name').drop('tone_id', 1)
    new = pd.concat([emotional_tone, language_tone, social_tone])
    new2 = new.T
    new2['Date'] = df['notes'].index[row]
    new2 = new2.set_index('Date')
    new2['notes'] = df['notes'][row]
    return new2


In [44]:
conversation_analysis_2 = pd.DataFrame()
for i in range (159, 714):
    if isinstance(df_data_3.iloc[i]['notes'], str) == True:
        conversation_analysis_2 = pd.concat([conversation_analysis_2, analyze_tones(df_data_3, i)])
    else:
        continue
    print(i)

159
160
161
162
163
164
165
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
244
245
246
247
248
249
250
251
252
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
338
339
340
341
342
343
344
345
346
347
348
349
350
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
382
383
384
385
386
387
388
389
390
391
392
394
395
396
397
398
400
401
402
403
404
405
407
408
410
411
412
413
414
415
416
417
418
419
420


IndexError: single positional indexer is out-of-bounds

In [45]:
conversation_analysis = pd.concat([conversation_analysis, conversation_analysis_2])

In [5]:
exercise_analysis = pd.DataFrame()
for i in range(len(df_data_2)):
    exercise_analysis = pd.concat([exercise_analysis, analyze_tones(df_data_2, i)])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214


In [25]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

#df = pd.DataFrame(data = [[1,2],[3,4]], columns=['Col 1', 'Col 2'])
create_download_link(exercise_analysis)

In [47]:
create_download_link(conversation_analysis)